In [1]:
# Import useful libraries
import pandas as pd, sys
sys.path.append('../3-ml-models')
import utils

# Generalization

The project aim is to create a Machine Learning model capable of detecting the dates when a crop field has been manured, using satellite data. <br>
Now is the turn to see how much the *best model*, obtained in the [ML models notebook](https://github.com/Amatofrancesco99/master-thesis/blob/main/Notebooks/3-ml-models/notebook.ipynb), can generalize our classification problem. More precisely, the purpose is to see how it performs in detecting when crop fields have been manured in a different country from the trained one.

**N.B:** Since using radar indexes is useless (as seen), they won't be considered in this notebook.

In [2]:
s2_df = pd.read_csv('../../Datasets/other/other-fields-s2-features-extracted.gz', compression='gzip')

## Modify original DataFrame

The purpose of building these `DataFrame`, modifying the original one, is to have them structured in the same manner used to train the model.

In [3]:
# Sentinel-2
s2_df_mod = utils.get_modified_df(s2_df, sentinel=2)
s2_df_mod

,crop_field_name,consequent_s2_acquisitions,B1,B2,B3,B4,B5,B6,B7,B8,...,CARI2,MCARI,MCARI1,MCARI2,BSI,GLI,ALTERATION,SDI,manure_dates,y
0,P-PVPR,"[2022-01-06, 2022-01-11]",687.810860,676.196793,381.072886,298.325802,11.222668,367.789723,483.986152,578.134475,...,399.462340,483.668123,1133.017347,0.203651,34.159906,0.106621,0.303743,526.808673,['2022-03-31'],0.0
1,P-PVPR,"[2022-01-11, 2022-01-16]",261.150510,330.489431,214.768586,182.770044,147.293003,72.776603,78.505102,18.978863,...,669.220964,117.772302,185.941574,0.052732,929.032119,0.062612,0.094897,328.088557,['2022-03-31'],0.0
2,P-PVPR,"[2022-01-16, 2022-01-31]",5771.107507,5180.021137,4761.511297,4624.255102,4369.051020,3827.163994,3641.225948,3586.435496,...,11036.881242,394.195026,1280.340569,0.451321,16567.592337,0.056195,0.323921,383.844023,['2022-03-31'],0.0
3,P-PVPR,"[2022-01-31, 2022-02-20]",3302.066327,2910.313411,2534.679665,2389.765306,2258.928571,1998.626822,1916.246356,1900.370262,...,6168.708603,83.587040,478.662464,0.057273,8456.930442,0.005238,0.001145,800.884840,['2022-03-31'],0.0
4,P-PVPR,"[2022-02-20, 2022-02-25]",7861.017493,6927.431851,6130.450437,5886.305758,5448.576895,4412.897230,4103.556122,4059.272595,...,286.831577,423.258445,2250.062055,0.390796,19894.776295,0.036314,0.085591,1202.978863,['2022-03-31'],0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,P-VAPR2,"[2022-11-20, 2022-11-25]",120.413483,171.456180,97.725843,97.532584,90.388764,50.752809,22.871910,114.833708,...,310.016522,25.751388,305.505978,0.032809,327.932683,0.020257,0.003821,242.179775,['2022-02-24'],0.0
136,P-VAPR2,"[2022-11-25, 2022-11-27]",2182.548315,1798.456180,1738.838202,1880.164045,1811.215730,1101.968539,900.321348,874.256180,...,8085.240306,213.366634,1668.975640,0.354067,6308.994070,0.048263,0.403094,1592.712360,['2022-02-24'],0.0
137,P-VAPR2,"[2022-11-27, 2022-12-07]",2235.002247,1918.566292,1792.642697,1874.588764,1767.062921,1254.134831,1118.862921,1052.071910,...,7626.600529,253.122779,1312.260135,0.320294,6738.953440,0.052033,0.359573,1456.548315,['2022-02-24'],0.0
138,P-VAPR2,"[2022-12-07, 2022-12-17]",24.741573,5.382022,24.435955,24.653933,60.869663,215.595506,247.285393,172.804494,...,60.123112,140.540057,360.920360,0.039155,157.826992,0.015101,0.004364,167.593258,['2022-02-24'],0.0


## Test the best model